# PDL - Granite IO Processor Demo

The Prompt Declaration Language (PDL) is a YAML-based declarative approach to prompt programming, where prompts are at the forefront. PDL facilitates model chaining and tool use, abstracting away the plumbing necessary for such compositions, enables type checking of the input and output of models, and is based on LiteLLM to support a variety of model providers. PDL has been used with RAG, CoT, ReAct, and an agent for solving SWE-bench. PDL is [open-source](https://github.com/IBM/prompt-declaration-language) and works well with watsonx.ai and Granite models.

You can use PDL stand-alone or from a Python SDK or, as shown here, in a notebook via a notebook extension. In the cell output, model-generated text is rendered in green font, and tool-generated text is rendered in purple font.

In this notebook, we demonstrate how PDL is integrated with the [Granite IO Processor](https://github.com/ibm-granite/granite-io) framework, which enables a developer to transform how a user calls an IBM Granite model and how the output from the model is returned to the user. PDL uses granite-io as an alternative backend to LiteLLM. The following examples show how to call an Ollama Granite model via PDL and granite-io, how to extract hallucination scores and citations, and how to toggle the thinking control, which turns on reasoning.

First make sure you have Ollama installed and ollama serve is running, and that you have pulled the `granite3.2:2b` model.

In [ ]:
! pip install 'prompt-declaration-language[examples]'

In [ ]:
%load_ext pdl.pdl_notebook_ext

## Model call with granite-io

In PDL, the user specifies step-by-step the shape of data they want to generate. In the following, the `text` construct indicates a text block containing a prompt and a model call. Implicitly, PDL builds a background conversational context (list of role/content) which is used to make model calls. Each model call uses the context built so far as its input prompt.

In this example, we infer using the `granite3.2:2b` model on Ollama via `granite-io`. Note that the `platform` field can be omited.

In [ ]:
%%pdl --reset-context
text:
- "Hello!\n"
- model: "granite3.2:2b"
  platform: granite-io
  backend: openai

## Model call with thinking flag

In the following example, we pass the `thinking` flag to the model, which causes it to reason. This flag is passed to the Ollama model via the `granite-io` library, which shapes the prompt appropriately given the `thinking` flag.

In [ ]:
%%pdl --reset-context
text:
- |
  Find the fastest way for a seller to visit all the cities in their region
  >> Response:
- model: "granite3.2:2b"
  backend: openai
  parameters: 
    thinking: true
  modelResponse: outputs
- |
  >> Thoughts:
  ${ outputs.results[0].next_message.reasoning_content }


## Hallucination Score and Citations

In the following example, we pass the hallucination and citations controls to the model call.

In [ ]:
%%pdl --reset-context
defs:
  doc:
    data:
      text: |
        Audrey Faith McGraw (born September 21, 1967) is an American singer 
        and record producer. She is one of the most successful country artists 
        of all time, having sold more than 40 million albums worldwide. Hill is 
        married to American singer Tim McGraw, with whom she has recorded several duets. 
        Hill's first two albums, Take Me as I Am (1993) and It Matters to Me (1995), 
        were major successes and placed a combined three number ones on Billboard's 
        country charts. Hill's debut album was Take Me as I Am (1993); sales were strong, 
        buoyed by the chart success of "Wild One". Hill became the first female country 
        singer in 30 years to hold Billboard's number one position for four consecutive 
        weeks when "Wild One" managed the feat in 1994. Her version of "Piece of My Heart", 
        also went to the top of the country charts in 1994. The album sold a total of 
        3 million copies. Other singles from the album include "Take Me as I Am".  The recording 
        of Faith's second album was delayed by surgery to repair a ruptured blood vessel on 
        her vocal cords. It Matters to Me finally appeared in 1995 and was another 
        success, with the title track becoming her third number-one country single. 
        Several other top 10 singles followed, and more than 3 million copies of the 
        album were sold. The fifth single from the album, "I Can't Do That Anymore", 
        was written by country music artist Alan Jackson. Other singles from the album 
        include "You Can't Lose Me", "Someone Else's Dream", and "Let's Go to Vegas". 
        During this period, Hill appeared on the acclaimed PBS music program Austin City Limits.  
        In spring 1996, Hill began the Spontaneous Combustion Tour with country singer Tim McGraw. 
        At that time, Hill had recently become engaged to her former producer, Scott Hendricks, 
        and McGraw had recently broken an engagement. McGraw and Hill were quickly 
        attracted to each other and began an affair. After discovering that Hill was 
        pregnant with their first child, the couple married on October 6, 1996. The 
        couple have three daughters together: Gracie Katherine (born 1997), Maggie Elizabeth (born 1998) 
        and Audrey Caroline (born 2001). Since their marriage, Hill and McGraw have endeavored 
        never to be apart for more than three consecutive days.  After the release of It Matters to Me, 
        Hill took a three-year break from recording to give herself a rest from four years of touring
        and to begin a family with McGraw. During her break, she joined forces with her husband 
        for their first duet, "It's Your Love". The song stayed at number one for six weeks, 
        and won awards from both the Academy of Country Music and the Country Music Association. 
        Hill has remarked that sometimes when they perform the song together, 
        "it [doesn't] feel like anybody else was really watching."

text:
- Did Faith Hill take a break from recording after releasing her second album, It Matters to Me?
- model: "granite3.2:2b"
  backend: openai
  parameters:
    documents:
    - ${ doc }
    controls:
      hallucinations: true
      citations: true
  modelResponse: output
- "\nHallucinations:\n"
- for:  
    hallucination: ${ output.results[0].next_message.hallucinations }
  repeat:
    text: 
    - "Hallucination Risk: ${ hallucination.risk }"
    - "\nSentence: ${ hallucination.response_text }"
  join: 
    with: "\n"
- "\n\nCitations:\n"
- for:
    citation: ${ output.results[0].next_message.citations }
  repeat:
    text: 
    - "Citation: ${ citation.context_text }"
    - "\nSentence: ${ citation.response_text }"
  join:
    with: "\n"

## Conclusion

Since prompts are at the forefront, PDL makes users more productive in their trial-and-error with LLMs. With the `granite-io` platform, PDL users can take advantage of controls such as thinking, hallucination scores and citations. Try it!

https://github.com/IBM/prompt-declaration-language